In [1]:
import pandas as pd
import re
import requests
import json
import os

In [2]:
df =  pd.read_csv('atriographics_030225.csv', sep = ";", encoding="utf-8")
pd.set_option('display.max_colwidth', None)

In [3]:
lista = df["Descripción"].str.split().str[0]

In [4]:
sorted(lista.unique())

['(Sesiones',
 'Aceptar',
 'Añadida',
 'Borrado',
 'Cambio',
 'Cargado',
 'Creado',
 'Duración',
 'Error',
 'Generar',
 'Intento',
 'Limpiar',
 'Login',
 'Logout',
 'Nuevo',
 'Solicitud',
 'Visualizar']

In [5]:
len(lista.unique())

17

In [6]:
mapeo_eventos = {
    '(Sesiones': lambda x: "CONEXION" if re.search(r'\bconectada\b', x.lower()) else "DESCONEXION",
    'Aceptar': "ACEPTAR_PRIVACIDAD",       
    'Añadida': "AÑADIR_PIEZA",
    'Borrado': "BORRAR_DISEÑO",          
    'Cambio': "CAMBIO_CONTRASEÑA",              
    'Cargado': "CARGAR_DISENO",
    'Creado': "CREAR_DISENO",
    'Duración': "DURACION",             
    'Error': "ERROR_SISTEMA",          
    'Generar': lambda x: "GEN_PRESUPUESTO" if re.search(r'\bpresupuesto\b', x.lower()) else "GEN_PEDIDO",
    'Intento': "INTENTO_RESET",       
    'Limpiar': "LIMPIAR_DATOS",        
    'Login': "LOGIN",
    'Logout': "LOGOUT",
    'Nuevo': "NUEVO_LOGO",              
    'Solicitud': "SOLICITUD_RESET",            
    'Visualizar': lambda x: "VER_PRESUPUESTO" if re.search(r'\bpresupuesto\b', x.lower()) else "VER_PEDIDO",
}

In [7]:
def categorizar_evento(descripcion):
    descripcion = str(descripcion)
    primera_palabra = descripcion.split()[0]
    
    if primera_palabra in mapeo_eventos:
        if callable(mapeo_eventos[primera_palabra]):
            return mapeo_eventos[primera_palabra](descripcion)
        else:
            return mapeo_eventos[primera_palabra]
    else:
        return "OTRO"

In [8]:
df["Evento"] = df["Descripción"].apply(categorizar_evento)

In [9]:
df[df['Evento'] == 'GEN_PEDIDO'][['Descripción', 'Evento']].head(10)

,Descripción,Evento
986,Generar pedido pdf para el diseño con id: 86108 y fabricante Azor,GEN_PEDIDO
1052,Generar pedido pdf para el diseño con id: 86117 y fabricante Azor,GEN_PEDIDO
3995,Generar pedido pdf para el diseño con id: 65674 y fabricante Glicerio Chaves,GEN_PEDIDO
4345,Generar pedido pdf para el diseño con id: 85901 y fabricante Salcedo Mueble,GEN_PEDIDO
5195,Generar pedido pdf para el diseño con id: 87965 y fabricante Salcedo Mueble,GEN_PEDIDO
7736,Generar pedido pdf para el diseño con id: 87704 y fabricante Aparicio Donoso,GEN_PEDIDO
7739,Generar pedido pdf para el diseño con id: 87704 y fabricante Aparicio Donoso,GEN_PEDIDO
7744,Generar pedido pdf para el diseño con id: 87704 y fabricante Aparicio Donoso,GEN_PEDIDO
7754,Generar pedido pdf para el diseño con id: 87704 y fabricante Aparicio Donoso,GEN_PEDIDO
8653,Generar pedido pdf para el diseño con id: -1 y fabricante Salcedo Mueble,GEN_PEDIDO


In [10]:
def extraer_columnas(row):
    evento = row['Evento']
    desc = row['Descripción']
    if pd.isna(desc):
        return {}

    if evento == 'AÑADIR_PIEZA':
        return {
            'pieza': buscar(r'pieza ([\w-]+)', desc),
            'piezaid': buscar(r'id (\d+)', desc),
            'catalogo': buscar(r'catálogo (.*?) en el diseño', desc),
            'diseñoid': buscar(r'diseño con id: (-?\d+)', desc)
        }

    elif evento == 'LOGIN' or evento == 'LOGOUT' or evento == 'SOLICITUD_RESET' or evento == 'ACEPTAR_PRIVACIDAD' or evento == 'CAMBIO_CONTRASEÑA':
        return {
            'userid': buscar(r'userID: (\w+)', desc)
        }
    elif evento == 'GEN_PEDIDO':
        return {
            'diseñoid': buscar(r'diseño con id: (-?\d+)', desc),
            'fabricante': buscar(r'fabricante (.*)', desc)
        }

    elif evento == 'CREAR_DISENO' or evento == 'CARGAR_DISENO' or evento == 'GEN_PRESUPUESTO' or evento == 'VER_PRESUPUESTO' or evento == 'ERROR_SISTEMA' or evento == 'BORRAR_DISEÑO' or evento == 'VER_PEDIDO':
        return {
            'diseñoid': buscar(r'diseño con id: (-?\d+)', desc)
        }

    elif evento == 'INTENTO_RESET':
        return {
            'email': buscar(r'email[:\s]+(\S+)', desc)
        }

    elif evento == 'DURACION':
        return {
            'minutos': buscar(r'Duración de la sesión:\s*([\d.]+)', desc)
        }

    elif evento == 'NUEVO_LOGO':
        return {
            'logo': buscar(r'logo[:\s]+(\w+)', desc)
        }

    elif evento == 'CONEXION' or evento == 'DESCONEXION':
        return {
            'nsesiones': buscar(r'activas (\d+)', desc)
        }
    return {}
def buscar(patron, texto):
    match = re.search(patron, texto)
    return match.group(1) if match else None

In [11]:
df_extra = df.apply(extraer_columnas, axis=1, result_type='expand')

In [12]:
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y %H:%M', errors='coerce')

In [13]:
df = pd.concat([df, df_extra], axis=1)

In [14]:
def geolocalizar(ip):
    url= f'https://ipinfo.io/{ip}?token=aff46df76e5e05'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        lat_str, lng_str = data['loc'].split(',')
        lat, lng = float(lat_str), float(lng_str)
        return {
            'city': data.get('city'),
            'region': data.get('region'),
            'country': data.get('country'),
            'lat': lat,
            'lng': lng
        }
    except Exception as e:
        print(f'Error con la ip: {ip}')
        return {'city': None, 'region': None, 'country': None, 'lat': None, 'lng': None}

In [15]:
ips = df['IP'].unique()

In [16]:
geo = 'geolocalizacion.json'

if os.path.exists(geo) and os.path.getsize(geo) > 0:
    with open(geo, 'r') as f:
        geolocalizacion = json.load(f)
else:
    geolocalizacion = {}


In [17]:
for ip in ips:
    if ip not in geolocalizacion:
        geolocalizacion[ip] = geolocalizar(ip) 
with open('geolocalizacion.json', 'w') as f:
    json.dump(geo, f)

In [18]:
with open('geolocalizacion.json', 'w') as f:
    json.dump(geolocalizacion, f)

In [19]:
df['city'] = df['IP'].map(lambda ip: geolocalizacion.get(ip, {}).get('city'))
df['region'] = df['IP'].map(lambda ip: geolocalizacion.get(ip, {}).get('region'))
df['country'] = df['IP'].map(lambda ip: geolocalizacion.get(ip, {}).get('country'))
df['lat'] = df['IP'].map(lambda ip: geolocalizacion.get(ip, {}).get('lat'))
df['lng'] = df['IP'].map(lambda ip: geolocalizacion.get(ip, {}).get('lng'))
df

,Fecha,Usuario,ID,Acceso,Descripción,IP,Evento,nsesiones,pieza,piezaid,...,userid,minutos,fabricante,logo,email,city,region,country,lat,lng
0,2025-02-03 14:01:00,#0 - null,4466470,CONN,(Sesiones activas 56) Sesión conectada,-1,CONEXION,56,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN
1,2025-02-03 14:01:00,#13856 - COLCHONERIA 3000. S.L.,4466469,OK,Añadida la pieza M10749 con id 419579 del catálogo APARICIO DONOSO - Explora 2.0 en el diseño con id: 88756,86.106.2.231,AÑADIR_PIEZA,NaN,M10749,419579,...,NaN,NaN,NaN,NaN,NaN,Valencia,Valencia,ES,39.4739,-0.3797
2,2025-02-03 14:01:00,#2708 - GRUPO HERVI FUENTE ALAMO SL,4466468,OK,Añadida la pieza B1207 con id 30515 del catálogo MUEBLES MÁGINA - Kloset en el diseño con id: 88755,83.138.40.18,AÑADIR_PIEZA,NaN,B1207,30515,...,NaN,NaN,NaN,NaN,NaN,Valencia,Valencia,ES,39.4739,-0.3797
3,2025-02-03 14:01:00,#15274 - DIJUVE,4466467,OK,Visualizar presupuesto para el diseño con id: 88431,83.48.121.13,VER_PRESUPUESTO,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Donostia / San Sebastián,Basque Country,ES,43.3128,-1.9750
4,2025-02-03 14:01:00,#0 - null,4466466,CONN,(Sesiones activas 55) Sesión desconectada,-1,DESCONEXION,55,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74302,2025-01-20 12:55:00,#1307 - LOMAR MUEBLISTAS,4392168,OK,Añadida la pieza COR501 con id 251044 del catálogo JGORBE - Oficina en el diseño con id: 86813,84.79.27.22,AÑADIR_PIEZA,NaN,COR501,251044,...,NaN,NaN,NaN,NaN,NaN,Villarrobledo,Castille-La Mancha,ES,39.2699,-2.6012
74303,2025-01-20 12:55:00,#7245 - MUEBLES LOZANO,4392167,OK,Cargado el diseño con id: 85875,89.29.219.252,CARGAR_DISENO,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Elda,Valencia,ES,38.4778,-0.7916
74304,2025-01-20 12:55:00,#7 - MUEBLES MARQUEZ BAENA S.C,4392166,OK,Añadida la pieza 604040 con id 214853 del catálogo RODRIGO - Kroma 6 en el diseño con id: -1,62.37.68.102,AÑADIR_PIEZA,NaN,604040,214853,...,NaN,NaN,NaN,NaN,NaN,Benalmádena,Andalusia,ES,36.5961,-4.5727
74305,2025-01-20 12:55:00,#3137 - PROGRAMAKLOSET,4392165,OK,Añadida la pieza EF120 con id 35005 del catálogo MUEBLES MÁGINA - Kloset en el diseño con id: -1,89.6.246.37,AÑADIR_PIEZA,NaN,EF120,35005,...,NaN,NaN,NaN,NaN,NaN,Madrid,Madrid,ES,40.4165,-3.7026


In [20]:
tipo_numerico = ['minutos']
df[tipo_numerico] = df[tipo_numerico].apply(pd.to_numeric, errors='coerce')

In [25]:
df['Hora'] = df['Fecha'].dt.hour
df['NombreDia'] = df['Fecha'].dt.day_name()

In [21]:
tipo_int = ['nsesiones', 'piezaid', 'diseñoid', 'userid']
df[tipo_int] = df[tipo_int].apply(lambda x: x.astype('Int64'))

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74307 entries, 0 to 74306
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Fecha        74307 non-null  datetime64[ns]
 1   Usuario      74307 non-null  object        
 2   ID           74307 non-null  int64         
 3   Acceso       74307 non-null  object        
 4   Descripción  74307 non-null  object        
 5   IP           74307 non-null  object        
 6   Evento       74307 non-null  object        
 7   nsesiones    15323 non-null  Int64         
 8   pieza        32807 non-null  object        
 9   piezaid      32807 non-null  Int64         
 10  catalogo     32807 non-null  object        
 11  diseñoid     42825 non-null  Int64         
 12  userid       10979 non-null  Int64         
 13  minutos      4893 non-null   float64       
 14  fabricante   86 non-null     object        
 15  logo         7 non-null      object        
 16  emai

In [27]:
df.to_parquet("logs.parquet",index=False)

In [28]:
df.to_csv("logs.csv",index=False)